In [14]:
%load_ext autoreload
%autoreload 2
%run ./initializer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [15]:
import pandas as pd
from lgbm_ranker import LGBMRankerModel, LGBMRanker
from feature_tester import FeatureTester
import numpy as np

In [16]:
train_df = pd.read_parquet("data/training_set_processed.parquet")
test_df = pd.read_parquet("data/test_set_processed.parquet")

In [17]:
params_fixed = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [5],
    "boosting_type": "gbdt",
    "n_estimators": 10000,
}
params_initial = {
    "learning_rate": 0.05,
}
params_small_hpo = {
    "learning_rate": 0.03427673821049949,
    "num_leaves": 79,
    "min_child_samples": 50,
    "reg_alpha": 0.0001416721648867782,
    "reg_lambda": 0.12245976197134716,
}
params_hpo = {
    "n_estimators": 1500,
    'learning_rate': 0.03928543917526321, 
    'num_leaves': 151, 
    'min_child_samples': 100, 
    'min_child_weight': 9.84223213747703, 
    'colsample_bytree': 0.5340379911098179, 
    'subsample': 0.9940711682871947, 
    'reg_alpha': 0.009867816340998009, 
    'reg_lambda': 8.490860770966702
    }

params = {**params_fixed, **params_hpo}
lgbm_class_model = LGBMRankerModel(train_df, params)

In [5]:
X_train, y_train, X_val, y_val, groups_size_train, groups_size_val = lgbm_class_model.format_data(train_df)

In [6]:
fitted_model = lgbm_class_model.fit(X_train, y_train, X_val, y_val, groups_size_train, groups_size_val)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.357467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8839
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 127
Training until validation scores don't improve for 500 rounds
[250]	valid_0's ndcg@5: 0.403682
[500]	valid_0's ndcg@5: 0.407004
[750]	valid_0's ndcg@5: 0.407576
[1000]	valid_0's ndcg@5: 0.406955
Early stopping, best iteration is:
[683]	valid_0's ndcg@5: 0.407833


In [7]:
lgbm_class_model.save_final_results(test_df)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Final results saved to final_predictions
         srch_id  prop_id
23             1    99484
9              1    54937
12             1    61934
5              1    28181
6              1    34263
...          ...      ...
4959177   332787    29018
4959182   332787    99509
4959181   332787    94437
4959178   332787    32019
4959180   332787    35240

[4959183 rows x 2 columns]


In [18]:
k_fold_predictions = lgbm_class_model.perform_k_fold_fit_and_predict(train_df, test_df)

Fitting model for fold 1 of 5


/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.312347 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8839
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 127
Training until validation scores don't improve for 500 rounds
[250]	valid_0's ndcg@5: 0.403682
[500]	valid_0's ndcg@5: 0.407004
[750]	valid_0's ndcg@5: 0.407576
[1000]	valid_0's ndcg@5: 0.406955
Early stopping, best iteration is:
[683]	valid_0's ndcg@5: 0.407833


/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Fitting model for fold 2 of 5


...

Fitting model for fold 5 of 5


/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.341830 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8850
[LightGBM] [Info] Number of data points in the train set: 3966678, number of used features: 127
Training until validation scores don't improve for 500 rounds
[250]	valid_0's ndcg@5: 0.404947
[500]	valid_0's ndcg@5: 0.408723
[750]	valid_0's ndcg@5: 0.409824
[1000]	valid_0's ndcg@5: 0.410456
[1250]	valid_0's ndcg@5: 0.409738
Early stopping, best iteration is:
[878]	valid_0's ndcg@5: 0.410694


/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Mean score for all 5 folds: 0.40973676157873246


In [19]:
final_df = k_fold_predictions[['srch_id', 'prop_id']]

In [21]:
import os
path = "data/final_predictions.csv"
if os.path.exists(path):
            os.remove(path)
final_df.to_csv(path, index=False)